# 2. Text Vectorization in TensorFlow

In [ ]:
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds

# Loading training data
imdb_data = tfds.load(name="imdb_reviews", split="train")
imdb_df = tfds.as_dataframe(imdb_data)
imdb_df['text'] = imdb_df['text'].str.decode('utf-8')
imdb_sample = imdb_df.sample(frac=0.2, random_state=100)

X_train = imdb_sample['text']
y_train = imdb_sample['label']

In [ ]:
# Loading testing data
imdb_test = tfds.load(name="imdb_reviews", split="test")
imdb_test_df = tfds.as_dataframe(imdb_test)
imdb_test_df['text'] = imdb_test_df['text'].str.decode('utf-8')
imdb_test_sample = imdb_test_df.sample(frac=0.2, random_state=100)

X_test = imdb_test_sample['text']
y_test = imdb_test_sample['label']

In [ ]:
# Target distribution
imdb_test_sample['label'].value_counts()

1    2506
0    2494
Name: label, dtype: int64

In [ ]:
# Check for missing values
imdb_test_sample.isna().sum()

label    0
text     0
dtype: int64

# 3. Creating the TextVectorizer Layer

In [ ]:
from tensorflow.keras.layers import TextVectorization

max_tokens = 25000
output_sequence_length = 256

vectorizer_layer = TextVectorization(max_tokens=max_tokens,
                                     output_mode='int',
                                     standardize='lower',
                                     output_sequence_length=output_sequence_length)

vectorizer_layer.adapt(X_train)

# 4. Building the Model with the Vectorized Layer

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, layers

model = Sequential()
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vectorizer_layer)
model.add(layers.Dense(16))
model.add(layers.Dense(1, activation='sigmoid'))

opt = tf.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {test_acc}")

Epoch 1/10
157/157 [==============================] - 1s 2ms/step - loss: 690.5369 - accuracy: 0.5084
Epoch 2/10
157/157 [==============================] - 0s 2ms/step - loss: 32.9394 - accuracy: 0.4984
Epoch 3/10
157/157 [==============================] - 0s 2ms/step - loss: 27.8380 - accuracy: 0.5162
Epoch 4/10
157/157 [==============================] - 0s 2ms/step - loss: 28.9621 - accuracy: 0.5004
Epoch 5/10
157/157 [==============================] - 0s 2ms/step - loss: 25.2356 - accuracy: 0.4980
Epoch 6/10
157/157 [==============================] - 0s 2ms/step - loss: 21.2950 - accuracy: 0.5152
Epoch 7/10
157/157 [==============================] - 0s 2ms/step - loss: 19.9154 - accuracy: 0.5170
Epoch 8/10
157/157 [==============================] - 0s 2ms/step - loss: 16.9722 - accuracy: 0.5146
Epoch 9/10
157/157 [==============================] - 0s 2ms/step - loss: 13.6325 - accuracy: 0.5040
Epoch 10/10
157/157 [==============================] - 0s 2ms/step - loss: 13.2806 - accur

# 5. Create an Embedding Layer

In [ ]:
output_dim = 128

from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(input_dim=max_tokens,
                            output_dim=output_dim,
                            input_length=output_sequence_length)

# 6. Text Classification Model with Embedding Layer

In [ ]:
vectorizer_layer = TextVectorization(max_tokens=max_tokens,
                                     output_mode='int',
                                     standardize='lower_and_strip_punctuation',
                                     ngrams=(1, 2),
                                     output_sequence_length=output_sequence_length)

vectorizer_layer.adapt(X_train)
embedding_layer = Embedding(input_dim=max_tokens,
                            output_dim=output_dim,
                            input_length=output_sequence_length)

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorizer_layer)
model.add(embedding_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

opt = tf.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {test_acc}")

Epoch 1/10
157/157 [==============================] - 6s 33ms/step - loss: 0.6874 - accuracy: 0.5340
Epoch 2/10
157/157 [==============================] - 5s 33ms/step - loss: 0.6791 - accuracy: 0.5498
Epoch 3/10
157/157 [==============================] - 5s 33ms/step - loss: 0.6744 - accuracy: 0.5570
Epoch 4/10
157/157 [==============================] - 4s 28ms/step - loss: 0.6724 - accuracy: 0.5599
Epoch 5/10
157/157 [==============================] - 5s 29ms/step - loss: 0.6717 - accuracy: 0.5588
Epoch 6/10
157/157 [==============================] - 5s 32ms/step - loss: 0.6711 - accuracy: 0.5582
Epoch 7/10
157/157 [==============================] - 4s 29ms/step - loss: 0.6704 - accuracy: 0.5608
Epoch 8/10
157/157 [==============================] - 5s 33ms/step - loss: 0.6701 - accuracy: 0.5606
Epoch 9/10
157/157 [==============================] - 5s 29ms/step - loss: 0.6696 - accuracy: 0.5615
Epoch 10/10
157/157 [==============================] - 1s 8ms/step - loss: 0.6998 - accurac

[0.6997720003128052, 0.5326390862464905]